In [4]:
import openai

openai.ChatCompletion.create(
  model="gpt-3.5-turbo-0613",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

<OpenAIObject chat.completion id=chatcmpl-7RAa0YmRsy2dJgf9oCXaDKs0V1W1A at 0x1bcfec559e0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The 2020 World Series was played at a neutral location due to the COVID-19 pandemic. The games were held at Globe Life Field in Arlington, Texas.",
        "role": "assistant"
      }
    }
  ],
  "created": 1686709880,
  "id": "chatcmpl-7RAa0YmRsy2dJgf9oCXaDKs0V1W1A",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 33,
    "prompt_tokens": 53,
    "total_tokens": 86
  }
}

In [46]:
import openai
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

def book_dinner(restaurant, day, num_ppl=2):
    """ book dinner on a given day for given number of people at given restaurant"""
    booking = {
        "restaurant": restaurant,
        "day": "June 20, 2023",
        "num_ppl": num_ppl
    }
    return json.dumps(booking)


def send_comfirmation(email, confirmation_msg):
    """ send a confirmation message to give email"""
    comfirmation_msg = {
        "email": "chatgpt@openai.com",
        "confirmation_msg": "booking confirmed"
    }
    return json.dumps(confirmation_msg)

def get_response(messages):
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo-0613",
        messages = messages,
        functions=[
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "day": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
            {
                "name": "book_dinner",
                "description": "book dinner on a given day for given number of people at given restaurant",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "restaurant": {
                            "type": "string",
                            "description": "The restaurant name, e.g. McDonalds",
                        },
                        "day": {"type": "string", "enum": ["06/20/2023", "06/21/2023"]},
                        "num_ppl": {"type": "integer", "description": "number of people for booking"},
                    },
                    "required": ["restaurant","day","num_ppl"],
                },
            },
            {
                "name": "send_comfirmation",
                "description": "send a comfirmation message to given email",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "email": {
                            "type": "string",
                            "description": "email address for sending comfirmation",
                        },
                        "confirmation_msg": {"type": "string", "description": "comfirmation message for booking"},
                    },
                    "required": ["email","confirmation_msg"],
                },
            },
            
        ],
        function_call="auto",
    )
    return response

In [84]:
prompt = "check the weather, if good weather then make a reservation at McDondadls in Toronto for 10 people on June 20, 2023"
messages = [{"role":"user", "content": prompt}]
while True:
    response = get_response(messages)
    print(response)
    if response['choices'][0]['finish_reason'] == 'function_call':
        function_name = response['choices'][0]['message']['function_call']['name']
        function_input_arguments = response['choices'][0]['message']['function_call']['arguments']
        if function_name == 'get_current_weather':
            arguments_raw = response['choices'][0]['message']['function_call']['arguments']
            arguments = json.loads(arguments_raw)['location']
            print(arguments)
                                  
            function_result =get_current_weather(arguments)
            
            messages.append(
                {
                'role':'assistant',
                'content': None,
                'function_call' :{
                    'arguments':arguments_raw,
                    'name': 'get_current_weather'}
                })
            messages.append(
                {'role': 'function',
                 'name':'get_current_weather',
                 'content': function_result}
            
            )
        elif function_name == 'book_dinner':
            arguments_raw = response['choices'][0]['message']['function_call']['arguments']
            restaurant = json.loads(arguments_raw)['restaurant']
            day = json.loads(arguments_raw)['day']
            num_ppl = json.loads(arguments_raw)['num_ppl']
                                  
            function_result =book_dinner(restaurant, day, num_ppl)
            
            messages.append(
                {
                'role':'assistant',
                'content': None,
                'function_call' :{
                    'arguments':arguments_raw,
                    'name': 'book_dinner'}
                })
            messages.append(
                {'role': 'function',
                 'name':'book_dinner',
                 'content': function_result}
            )
        elif function_name == 'send_comfirmation':
            arguments_raw = response['choices'][0]['message']['function_call']['arguments']
            email = json.loads(arguments_raw)['email']
            confirmation_msg = json.loads(arguments_raw)['confirmation_msg']

            function_result =send_comfirmation(email, confirmation_msg)
            
            messages.append(
                {
                'role':'assistant',
                'content': None,
                'function_call' :{
                    'arguments':arguments_raw,
                    'name': 'send_comfirmation'}
                })
            messages.append(
                {'role': 'function',
                 'name':'send_comfirmation',
                 'content': function_result}
            )
        response = get_response(messages)
                
    elif response['choices'][0]['finish_reason'] == 'stop':
        break

{
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n  \"location\": \"Toronto\"\n}",
          "name": "get_current_weather"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1686782891,
  "id": "chatcmpl-7RTZbGh4UsNj8NIveL1ZP3Lxe5P3B",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 229,
    "total_tokens": 245
  }
}
Toronto
{
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n  \"restaurant\": \"McDonald's\",\n  \"day\": \"06/20/2023\",\n  \"num_ppl\": 10\n}",
          "name": "book_dinner"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1686782893,
  "id": "chatcmpl-7RTZdbXZYRVWNpPrFTc4uCBFbutUn",
  "model": "gpt

In [72]:
get_current_weather(function_input_arguments)

'{"location": "{\\n  \\"location\\": \\"Toronto\\"\\n}", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [22]:
import openai
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

# Step 1, send model the user query and what functions it has access to
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]
    # Step 2, check if the model wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]

        # Step 3, call the function
        # Note: the JSON response from the model may not be valid JSON
        function_response = get_current_weather(
            location=message.get("location"),
            unit=message.get("unit"),
        )
        
        print('.........')
        print(message)
        print('.........')
        # Step 4, send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What is the weather like in boston?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

print(run_conversation())

.........
{
  "content": null,
  "function_call": {
    "arguments": "{\n  \"location\": \"Boston, MA\"\n}",
    "name": "get_current_weather"
  },
  "role": "assistant"
}
.........
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The current weather in Boston is sunny and windy, with a temperature of 72\u00b0F.",
        "role": "assistant"
      }
    }
  ],
  "created": 1686720014,
  "id": "chatcmpl-7RDDSW5yzJCSGlgQ90XzZKI06SRXX",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 70,
    "total_tokens": 88
  }
}


In [55]:
locals()['get_current_weather']('toronto')

'{"location": "toronto", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'